In [1]:
! python3 dataset_nmr.py
import torch
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
from model import Transformer
from scipy import interpolate

label = np.load("Dataset/Noise_data_nmr/label.npy")
noise_signals = np.load("Dataset/Noise_data_nmr/noise_signals.npy")

Begin to generate simulation signals
Save successfully


/mnt/DATA1/wuliubin/miniconda3/envs/AcDpILT/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X = np.linspace(0, 14, 140)
Y = np.arange(300)
X,Y = np.meshgrid(X, Y)

with torch.no_grad():
        module = Transformer.load_from_checkpoint("Result/DOSY/last.ckpt")
        module.cpu()
        module.eval()
        module.freeze()

Z_out = np.zeros([5, 300, 140])

for i in range(4):
        test_input = noise_signals[i, np.newaxis]
        
        test_input = torch.tensor(test_input)
        test_input = test_input.to(torch.float32)

        test_out = module(test_input)/3

        test_out = test_out * test_input[:, :, 0][:, :, np.newaxis]
        test_out = test_out[0].cpu().detach().numpy()
        # test_out[test_out < np.tile(((np.max(test_out, axis=1)) * 0.7)[:, np.newaxis], [1, module.hparams.label_size])] = 0
        Z_out[i] = test_out


Z_out[4] = label[0]

scio.savemat("Result/noise_immunity_result.mat", {"Z":Z_out})